In [ ]:
pip install -r requirements.txt

In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.applications import ResNet50

# Đường dẫn tới dữ liệu
image_dir = "/content/drive/MyDrive/disaster/Comprehensive Disaster Dataset(CDD)"

# Kích thước ảnh chuẩn hóa
IMG_SIZE = 128  # Kích thước chuẩn hóa ảnh về 128x128

# Danh sách lưu đường dẫn ảnh và nhãn
image_paths = []
labels = []

# Lấy danh sách các nhãn (labels) từ thư mục
all_labels = os.listdir(image_dir)

# Duyệt qua các thư mục cha (Non_Damage và Disaster)
for label in all_labels:
    label_path = os.path.join(image_dir, label)
    if os.path.isdir(label_path):  # Kiểm tra nếu là thư mục
        if label == "Non_Damage":
            # Duyệt qua tất cả các ảnh trong thư mục "Non_Damage"
            for image_name in os.listdir(label_path):
                image_paths.append(os.path.join(label_path, image_name))
                labels.append(0)  # Gán nhãn "Non Damage"
        elif label == "Disaster":
            # Duyệt qua tất cả các ảnh trong thư mục "Disaster"
            for image_name in os.listdir(label_path):
                image_paths.append(os.path.join(label_path, image_name))
                labels.append(1)  # Gán nhãn "Disaster"

# Chuyển nhãn thành numpy array
labels = np.array(labels)

# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(image_paths, labels, test_size=0.2, random_state=42)

# Tiền xử lý ảnh: Chuyển ảnh thành mảng numpy, thay đổi kích thước và chuẩn hóa
def preprocess_images(image_paths, labels):
    images = []
    filtered_labels = []  # List to hold corresponding labels for valid images
    error_count = 0
    for path, label in zip(image_paths, labels):
        try:
            img = Image.open(path)
            img = img.resize((IMG_SIZE, IMG_SIZE))  # Đảm bảo kích thước ảnh
            if img.mode != 'RGB':
                img = img.convert('RGB')  # Chuyển đổi ảnh thành RGB nếu không phải
            img = np.array(img)
            if img.shape == (IMG_SIZE, IMG_SIZE, 3):
                images.append(img)
                filtered_labels.append(label)
            else:
                error_count += 1
        except Exception as e:
            error_count += 1

    if error_count > 0:
        print(f"{error_count} images were skipped due to errors.")

    return np.array(images), np.array(filtered_labels)

# Tiền xử lý ảnh
X_train, y_train = preprocess_images(X_train, y_train)
X_test, y_test = preprocess_images(X_test, y_test)

# Chuẩn hóa dữ liệu
X_train = X_train / 255.0
X_test = X_test / 255.0

# Chuyển nhãn thành dạng one-hot encoding
y_train = to_categorical(y_train, 2)  # Chỉ có 2 lớp (Non Damage, Disaster)
y_test = to_categorical(y_test, 2)

# Tạo đối tượng ImageDataGenerator để tăng cường dữ liệu
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Fit dữ liệu tăng cường trên tập huấn luyện
datagen.fit(X_train)

# Sử dụng mô hình ResNet50 đã được huấn luyện trước (pretrained)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Đóng băng các lớp của mô hình cơ bản
base_model.trainable = False

# Xây dựng mô hình CNN mới với các lớp bổ sung
model = Sequential([
    base_model,  # Thêm mô hình đã huấn luyện trước
    GlobalAveragePooling2D(),  # Global pooling thay vì Flatten
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')  # Chỉ có 2 lớp (Non Damage, Disaster)
])

# Biên dịch mô hình
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Sử dụng EarlyStopping để tránh overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Tạo callback để lưu mô hình tốt nhất dựa trên độ chính xác xác nhận (val_accuracy)
checkpoint = ModelCheckpoint("best_model.keras", monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

# Huấn luyện mô hình
model.fit(datagen.flow(X_train, y_train, batch_size=32),
          validation_data=(X_test, y_test),
          epochs=50,
          callbacks=[early_stopping, checkpoint])

# Dự đoán trên tập kiểm tra
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)

# In ma trận nhầm lẫn
print(confusion_matrix(np.argmax(y_test, axis=1), y_pred_labels))

# In báo cáo phân loại
print("\nBáo cáo phân loại:")
print(classification_report(np.argmax(y_test, axis=1), y_pred_labels, target_names=["Non Damage", "Disaster"]))


1 images were skipped due to errors.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


191/192 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step - accuracy: 0.6674 - loss: 0.6457
Epoch 1: val_accuracy improved from -inf to 0.66884, saving model to best_model.keras
192/192 ━━━━━━━━━━━━━━━━━━━━ 53s 209ms/step - accuracy: 0.6675 - loss: 0.6455 - val_accuracy: 0.6688 - val_loss: 0.6257
Epoch 2/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - accuracy: 0.6871 - loss: 0.6206
Epoch 2: val_accuracy improved from 0.66884 to 0.66949, saving model to best_model.keras
192/192 ━━━━━━━━━━━━━━━━━━━━ 65s 152ms/step - accuracy: 0.6872 - loss: 0.6206 - val_accuracy: 0.6695 - val_loss: 0.6170
Epoch 3/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - accuracy: 0.6943 - loss: 0.6131
Epoch 3: val_accuracy did not improve from 0.66949
192/192 ━━━━━━━━━━━━━━━━━━━━ 40s 149ms/step - accuracy: 0.6942 - loss: 0.6131 - val_accuracy: 0.6695 - val_loss: 0.6068
Epoch 4/50
191/192 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step - accuracy: 0.6863 - loss: 0.6119
Epoch 4: val_accuracy did not improve from 0.66949
192/192 ━━━━━━━━━━━━━━

In [ ]:
import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Đường dẫn tới mô hình đã lưu
model_path = "best_model.keras"  # Đường dẫn tới mô hình bạn đã lưu

# Tải mô hình đã huấn luyện
model = load_model(model_path)

# Kích thước ảnh chuẩn hóa
IMG_SIZE = 128  # Chỉnh kích thước ảnh về 128x128

# Tiền xử lý ảnh mới (cũng giống như tiền xử lý khi huấn luyện)
def preprocess_image(img_path):
    img = Image.open(img_path)
    img = img.resize((IMG_SIZE, IMG_SIZE))  # Đảm bảo kích thước ảnh
    if img.mode != 'RGB':
        img = img.convert('RGB')  # Chuyển đổi ảnh thành RGB nếu không phải
    img = np.array(img)

    # Chuẩn hóa dữ liệu
    img = img / 255.0

    # Thêm chiều mới để giống như dữ liệu đầu vào (batch size = 1)
    img = np.expand_dims(img, axis=0)

    return img

# Dự đoán cho một ảnh mới
def predict_single_image(img_path):
    # Tiền xử lý ảnh
    img = preprocess_image(img_path)

    # Dự đoán nhãn của ảnh
    prediction = model.predict(img)
    predicted_label = np.argmax(prediction, axis=1)

    # In kết quả dự đoán
    print(f"Dự đoán nhãn: {'Disaster' if predicted_label == 0 else 'Non Damage'}")

# Dự đoán cho một ảnh mới (thay đổi đường dẫn ảnh)
img_path = "/content/drive/MyDrive/disaster/Comprehensive Disaster Dataset(CDD)/Non_Damage/06_01_0001.png"  # Thay bằng đường dẫn ảnh bạn muốn kiểm thử
predict_single_image(img_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Dự đoán nhãn: Non Damage
